In [ ]:
!nvidia-smi

Thu May  6 08:26:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    22W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Ensemble_0506/data_all.zip -d /content

In [ ]:
%cd /content/drive/MyDrive/Trash_seg_0506

/content/drive/MyDrive/Trash_seg_0506


In [ ]:
!pip install segmentation_models_pytorch
!pip install albumentations==0.5.2

## import

In [ ]:
import os
import random
import time
import json
import warnings 
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from utils import *
import cv2
from sklearn.model_selection import StratifiedKFold
import glob

import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import zipfile

# Pretrained Model
import segmentation_models_pytorch as smp

# torchvision Models
import torchvision
from torchvision import models
from torchvision.models.segmentation.deeplabv3 import DeepLabHead

# 전처리를 위한 라이브러리
from torch.utils.data.sampler import WeightedRandomSampler
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms

import albumentations as A
from albumentations.pytorch import ToTensorV2

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

plt.rcParams['axes.grid'] = False

print('pytorch version: {}'.format(torch.__version__))
print('GPU 사용 가능 여부: {}'.format(torch.cuda.is_available()))

print(torch.cuda.get_device_name(0))
print(torch.cuda.device_count())

device = "cuda" if torch.cuda.is_available() else "cpu"   # GPU 사용 가능 여부에 따라 device 정보 저장
%matplotlib inline

pytorch version: 1.8.1+cu101
GPU 사용 가능 여부: True
Tesla V100-SXM2-16GB
1


## 하이퍼파라미터 세팅 및 seed 고정

In [ ]:
batch_size = 10  # Mini-batch size
num_epochs = 15
learning_rate = 1e-4

In [ ]:
# seed 고정
random_seed = 42
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

## Custom Dataset

In [ ]:
class TrashDataset(Dataset):
    def __init__(self, df, root='/content/', mode="train", transform=None):
        self.df = df.reset_index(drop=True).copy()
        self.mode = mode
        self.transform = transform
        self.root = root
        self.mask_label=[]
        self.mask_image = []
        mask_path = self.root + self.df['masks']

        for m_path in tqdm(mask_path.tolist()):
            # print(m_path)
            masks = cv2.imread(m_path, cv2.IMREAD_GRAYSCALE).astype(np.float32)
            self.fold_label(masks)
    
    def fold_label(self, masks):
        len(list(np.unique(masks)))
        if 10.0 in list(np.unique(masks)): self.mask_label.append(0)
        elif 1.0 in list(np.unique(masks)): self.mask_label.append(1)
        elif 11.0 in list(np.unique(masks)): self.mask_label.append(2)
        elif 6.0 in list(np.unique(masks)): self.mask_label.append(3)
        elif 5.0 in list(np.unique(masks)): self.mask_label.append(4)
        elif 4.0 in list(np.unique(masks)): self.mask_label.append(5)
        elif 8.0 in list(np.unique(masks)): self.mask_label.append(6)
        elif 7.0 in list(np.unique(masks)): self.mask_label.append(7)
        elif 2.0 in list(np.unique(masks)): self.mask_label.append(8)
        elif 3.0 in list(np.unique(masks)): self.mask_label.append(9)
        elif 9.0 in list(np.unique(masks)): self.mask_label.append(10)
        else: self.mask_label.append(11)  
    
        self.mask_label.append(len(np.unique(masks)))
        self.mask_image.append(masks)


    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image_path = self.root + self.df.iloc[idx]['filepath']
        imgs = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        
        if self.mode=="train" or self.mode=="val":
            mask_path = self.root + self.df.iloc[idx]['masks']
            masks = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE).astype(np.float32)
            transformed = self.transform(image=imgs, mask=masks)
            imgs = transformed["image"]
            masks = transformed["mask"]
            return imgs, masks
        
        elif self.mode == "test":
            transformed = self.transform(image=imgs)
            imgs = transformed["image"]
            return imgs

In [ ]:
# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))

train_transform = A.Compose([
                            A.Rotate(border_mode=1, p=0.5),
                            A.ShiftScaleRotate(border_mode=1, p=0.5),
                            A.HorizontalFlip(p=0.5),
                            A.VerticalFlip(p=0.5),
                            A.Cutout(num_holes=4, max_h_size=20, max_w_size=20),
                            A.Normalize(
                                mean=(0.485, 0.456, 0.406),
                                std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0
                            ),
                            ToTensorV2(),
                            ])

test_transform = A.Compose([
                            A.Normalize(
                                mean=(0.485, 0.456, 0.406),
                                std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0
                            ),
                            ToTensorV2(),
                           ])


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Trash_seg_0506/train.csv") 

additional_df = pd.read_csv("/content/drive/MyDrive/Trash_seg_0506/test.csv")  
df = pd.concat([df, additional_df], ignore_index=True)
df.head()

,filepath,masks,objects
0,batch_01_vt/0002.jpg,batch_01_vt_masks/0002.png,"[0, 6, 7, 9]"
1,batch_01_vt/0003.jpg,batch_01_vt_masks/0003.png,"[0, 2, 9]"
2,batch_01_vt/0005.jpg,batch_01_vt_masks/0005.png,"[0, 7]"
3,batch_01_vt/0006.jpg,batch_01_vt_masks/0006.png,"[0, 5, 7]"
4,batch_01_vt/0007.jpg,batch_01_vt_masks/0007.png,"[0, 7]"


In [ ]:
train_dataset = TrashDataset(df, mode="train", transform=train_transform)

## Class Score

In [ ]:
def class_score(avrg_class_IoU):
    # Class Score
    class_name=['BG','UNK','General Trash','Paper','Paper pack','Metal','Glass','Plastic','Styrofoam','Plastic Bag','Battery','Clothing']
    print('-'*80)
    print('Validation Class Pred mIoU Score')
    for idx, class_score in enumerate(avrg_class_IoU):
        print('[{}] mIoU : [{:.4f}]'.format(class_name[idx],class_score))
    print('-'*80) 

## CutMix

In [ ]:
def rand_bbox(W, H, lam):
    cut_rat = torch.sqrt(1.0 - lam)
    cut_w = (W * cut_rat).type(torch.long)
    cut_h = (H * cut_rat).type(torch.long)
    # uniform
    cx = torch.randint(W, (1,)).to(device)
    cy = torch.randint(H, (1,)).to(device)
    x1 = torch.clamp(cx - cut_w // 2, 0, W)
    y1 = torch.clamp(cy - cut_h // 2, 0, H)
    x2 = torch.clamp(cx + cut_w // 2, 0, W)
    y2 = torch.clamp(cy + cut_h // 2, 0, H)
    return x1, y1, x2, y2

In [ ]:
def cutmix_data(x, y, alpha=1.0, p=0.3):
    # x-> img 
    # y-> mask
    if np.random.random() > p:
        return x, y
    W, H = x.size(2), x.size(3)
    shuffle = torch.randperm(x.size(0)).to(device)
    cutmix_x = x
    cutmix_y = y

    lam = torch.distributions.beta.Beta(alpha, alpha).sample().to(device)
    x1, y1, x2, y2 = rand_bbox(W, H, lam)
    cutmix_x[:, :, x1:x2, y1:y2] = x[shuffle, :, x1:x2, y1:y2]
    cutmix_y[:, x1:x2, y1:y2] = y[shuffle, x1:x2, y1:y2]
    # Adjust lambda to match pixel ratio
    #lam = 1 - ((x2 - x1) * (y2 - y1) / float(W * H)).item()
    return cutmix_x, cutmix_y

## train, validation, test 함수 정의

In [ ]:
def train(fold, num_epochs, model, data_loader, val_loader, criterion, optimizer, scheduler, device):
    print('-'*80)
    print(f'Fold : [{fold}] Start training..')
    print('-'*80)
    early_stop=EarlyStopping(patience=5,path='./saved/'+str(fold)+'_checkpoint.pt')

    for epoch in range(num_epochs):
        train_loss=[]
        model.train()
        for step, (images, masks) in enumerate(tqdm(data_loader)):
            images = torch.stack(images)       # (batch, channel, height, width)
            masks = torch.stack(masks).long()  # (batch, channel, height, width)

            # gpu 연산을 위해 device 할당
            images, masks = images.to(device), masks.to(device)

            #Cut-Mix
            images, masks = cutmix_data(images, masks)

            # inference
            outputs = model(images)

            # loss 계산
            loss = criterion(outputs, masks)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

            #Cosine Annealing Scheduler
            scheduler.step(epoch + step / len(data_loader))

        # validation 주기에 따른 loss 출력 및 best model 저장
        avrg_loss, avrg_IoU, avrg_class_IoU, pb_IoU = validation(model, val_loader, device)
        print('Epoch [{}/{}], Train Loss: {:.4f} Vali Loss: {:.4f}, Vali mIoU: {:.4f}, Vali Public mIoU: {:.4f}'.format(epoch+1, num_epochs, np.mean(train_loss), avrg_loss, avrg_IoU, pb_IoU))
        # Reduce Scheduler
        scheduler.step(avrg_IoU)
        # Class Score
        #class_score(avrg_class_IoU)
        # Save
        early_stop(avrg_IoU,model)
        if early_stop.early_stop:
            print('Stop Training.....')
            break

In [ ]:
def validation(model, data_loader, device):
    criterion = SoftCrossEntropyLoss(smooth_factor=0.1).to(device) 
    n_class=12
    hist = np.zeros((n_class, n_class))
    print('Start validation')
    model.eval()
    with torch.no_grad():
        total_loss = 0
        cnt = 0
        pb_mIoU_list = []
        for step, (images, masks) in enumerate(tqdm(data_loader)):
            images = torch.stack(images)       # (batch, channel, height, width)
            masks = torch.stack(masks).long()  # (batch, channel, height, width)

            images, masks = images.to(device), masks.to(device)        

            outputs = model(images)
            loss = criterion(outputs, masks)
            total_loss += loss
            cnt += 1
            
            outputs = torch.argmax(outputs, dim=1).detach().cpu().numpy()
            
            hist = add_hist(hist, masks.detach().cpu().numpy(), outputs, n_class=n_class)
            pb_mIoU = public_label_accuracy_score(masks.detach().cpu().numpy(), outputs, n_class=12)
            pb_mIoU_list.append(pb_mIoU)
        
        vali_mIoU, vali_class_mIoU = label_accuracy_score(hist)
        avrg_loss = total_loss / cnt

    return avrg_loss, vali_mIoU, vali_class_mIoU, np.mean(pb_mIoU_list)

## Label Smoothing

In [ ]:
from typing import Optional

def label_smoothed_nll_loss(
    lprobs: torch.Tensor, target: torch.Tensor, epsilon: float, ignore_index=None, reduction="mean", dim=-1
) -> torch.Tensor:
    """
    Source: https://github.com/pytorch/fairseq/blob/master/fairseq/criterions/label_smoothed_cross_entropy.py
    :param lprobs: Log-probabilities of predictions (e.g after log_softmax)
    :param target:
    :param epsilon:
    :param ignore_index:
    :param reduction:
    :return:
    """
    if target.dim() == lprobs.dim() - 1:
        target = target.unsqueeze(dim)

    if ignore_index is not None:
        pad_mask = target.eq(ignore_index)
        target = target.masked_fill(pad_mask, 0)
        nll_loss = -lprobs.gather(dim=dim, index=target)
        smooth_loss = -lprobs.sum(dim=dim, keepdim=True)

        # nll_loss.masked_fill_(pad_mask, 0.0)
        # smooth_loss.masked_fill_(pad_mask, 0.0)
        nll_loss = nll_loss.masked_fill(pad_mask, 0.0)
        smooth_loss = smooth_loss.masked_fill(pad_mask, 0.0)
    else:
        nll_loss = -lprobs.gather(dim=dim, index=target)
        smooth_loss = -lprobs.sum(dim=dim, keepdim=True)

        nll_loss = nll_loss.squeeze(dim)
        smooth_loss = smooth_loss.squeeze(dim)

    if reduction == "sum":
        nll_loss = nll_loss.sum()
        smooth_loss = smooth_loss.sum()
    if reduction == "mean":
        nll_loss = nll_loss.mean()
        smooth_loss = smooth_loss.mean()

    eps_i = epsilon / lprobs.size(dim)
    loss = (1.0 - epsilon) * nll_loss + eps_i * smooth_loss
    return loss

class SoftCrossEntropyLoss(nn.Module):

    __constants__ = ["reduction", "ignore_index", "smooth_factor"]

    def __init__(
        self,
        reduction: str = "mean",
        smooth_factor: Optional[float] = None,
        ignore_index: Optional[int] = -100,
        dim: int = 1,
    ):
        """Drop-in replacement for torch.nn.CrossEntropyLoss with label_smoothing
        
        Args:
            smooth_factor: Factor to smooth target (e.g. if smooth_factor=0.1 then [1, 0, 0] -> [0.9, 0.05, 0.05])
        
        Shape
             - **y_pred** - torch.Tensor of shape (N, C, H, W)
             - **y_true** - torch.Tensor of shape (N, H, W)
        Reference
            https://github.com/BloodAxe/pytorch-toolbelt
        """
        super().__init__()
        self.smooth_factor = smooth_factor
        self.ignore_index = ignore_index
        self.reduction = reduction
        self.dim = dim

    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        log_prob = F.log_softmax(y_pred, dim=self.dim)
        return label_smoothed_nll_loss(
            log_prob,
            y_true,
            epsilon=self.smooth_factor,
            ignore_index=self.ignore_index,
            reduction=self.reduction,
            dim=self.dim,
        )

In [ ]:
from typing import Optional

def label_smoothed_nll_loss(
    lprobs: torch.Tensor, target: torch.Tensor, epsilon: float, ignore_index=None, reduction="mean", dim=-1
) -> torch.Tensor:
    """
    Source: https://github.com/pytorch/fairseq/blob/master/fairseq/criterions/label_smoothed_cross_entropy.py
    :param lprobs: Log-probabilities of predictions (e.g after log_softmax)
    :param target:
    :param epsilon:
    :param ignore_index:
    :param reduction:
    :return:
    """
    if target.dim() == lprobs.dim() - 1:
        target = target.unsqueeze(dim)

    if ignore_index is not None:
        pad_mask = target.eq(ignore_index)
        target = target.masked_fill(pad_mask, 0)
        nll_loss = -lprobs.gather(dim=dim, index=target)
        smooth_loss = -lprobs.sum(dim=dim, keepdim=True)

        # nll_loss.masked_fill_(pad_mask, 0.0)
        # smooth_loss.masked_fill_(pad_mask, 0.0)
        nll_loss = nll_loss.masked_fill(pad_mask, 0.0)
        smooth_loss = smooth_loss.masked_fill(pad_mask, 0.0)
    else:
        nll_loss = -lprobs.gather(dim=dim, index=target)
        smooth_loss = -lprobs.sum(dim=dim, keepdim=True)

        nll_loss = nll_loss.squeeze(dim)
        smooth_loss = smooth_loss.squeeze(dim)

    if reduction == "sum":
        nll_loss = nll_loss.sum()
        smooth_loss = smooth_loss.sum()
    if reduction == "mean":
        nll_loss = nll_loss.mean()
        smooth_loss = smooth_loss.mean()

    eps_i = epsilon / lprobs.size(dim)
    loss = (1.0 - epsilon) * nll_loss + eps_i * smooth_loss
    return loss

class SoftCrossEntropyLoss(nn.Module):

    __constants__ = ["reduction", "ignore_index", "smooth_factor"]

    def __init__(
        self,
        reduction: str = "mean",
        smooth_factor: Optional[float] = None,
        ignore_index: Optional[int] = -100,
        dim: int = 1,
    ):
        """Drop-in replacement for torch.nn.CrossEntropyLoss with label_smoothing
        
        Args:
            smooth_factor: Factor to smooth target (e.g. if smooth_factor=0.1 then [1, 0, 0] -> [0.9, 0.05, 0.05])
        
        Shape
             - **y_pred** - torch.Tensor of shape (N, C, H, W)
             - **y_true** - torch.Tensor of shape (N, H, W)
        Reference
            https://github.com/BloodAxe/pytorch-toolbelt
        """
        super().__init__()
        self.smooth_factor = smooth_factor
        self.ignore_index = ignore_index
        self.reduction = reduction
        self.dim = dim

    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        log_prob = F.log_softmax(y_pred, dim=self.dim)
        return label_smoothed_nll_loss(
            log_prob,
            y_true,
            epsilon=self.smooth_factor,
            ignore_index=self.ignore_index,
            reduction=self.reduction,
            dim=self.dim,
        )


## 모델 생성 및 Loss function, Optimizer 정의

In [ ]:
# k-fold cross validation

folds = StratifiedKFold(n_splits=5, shuffle=True)
for current_fold,(train_idx, vali_idx) in enumerate(folds.split(train_dataset,train_dataset.mask_label)):
    model = smp.DeepLabV3Plus('resnext50_32x4d', encoder_weights="swsl", classes=12)
    model.eval()
    model.to(device)

    train_data=torch.utils.data.Subset(train_dataset,train_idx)
    vali_data=torch.utils.data.Subset(train_dataset,vali_idx)

    train_loader=DataLoader(train_data,
                            batch_size=batch_size,
                            shuffle=True,
                            collate_fn=collate_fn,
                            num_workers=0,
                            drop_last=True)
    val_loader=DataLoader(vali_data,
                          batch_size=1,
                          shuffle=False,
                          collate_fn=collate_fn,
                          num_workers=0)

    criterion = SoftCrossEntropyLoss(smooth_factor=0.1).to(device)
    # criterion = nn.CrossEntropyLoss().to(device)

    optimizer = torch.optim.Adam(params = model.parameters(), lr = learning_rate)
    # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=1,threshold_mode='abs',min_lr=1e-8, verbose=True)
    scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=20, eta_max=learning_rate,  T_up=2, gamma=0.5)
    
    train(current_fold, num_epochs, model, train_loader, val_loader, criterion, optimizer, scheduler, device)

Downloading: "https://dl.fbaipublicfiles.com/semiweaksupervision/model_files/semi_weakly_supervised_resnext50_32x4-72679e44.pth" to /root/.cache/torch/hub/checkpoints/semi_weakly_supervised_resnext50_32x4-72679e44.pth



--------------------------------------------------------------------------------
Fold : [2] Start training..
--------------------------------------------------------------------------------



Start validation



Epoch [1/40], Train Loss: 1.0094 Vali Loss: 0.3997, Vali mIoU: 0.3389, Vali Public mIoU: 0.5181



Start validation


...


Start validation



Epoch [22/40], Train Loss: 0.5811 Vali Loss: 0.2700, Vali mIoU: 0.6255, Vali Public mIoU: 0.6519
Epoch    22: reducing learning rate of group 0 to 3.1250e-06.



Start validation



Epoch [23/40], Train Loss: 0.5794 Vali Loss: 0.2707, Vali mIoU: 0.6292, Vali Public mIoU: 0.6534
Stop Training.....
--------------------------------------------------------------------------------
Fold : [4] Start training..
--------------------------------------------------------------------------------


KeyboardInterrupt: ignored

## Inference 

In [ ]:
class CustomDataLoader(Dataset):
    """COCO format"""
    def __init__(self, data_dir, mode = 'test', transform = None):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        
    def __getitem__(self, index: int):
        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=index)
        image_infos = self.coco.loadImgs(image_id)[0]
        
        # cv2 를 활용하여 image 불러오기
        images = cv2.imread(os.path.join('/content', image_infos['file_name']))
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB)
        
        if self.mode == 'test':
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]
            
            return images, image_infos
    
    
    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

In [ ]:
# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))

test_transform = A.Compose([
                            A.Normalize(
                                mean=(0.485, 0.456, 0.406),
                                std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0
                            ),
                            ToTensorV2(),
                           ])

In [ ]:
test_dataset = CustomDataLoader('/content/test.json', mode="test", transform=test_transform)

test_loader = DataLoader(test_dataset, 
                          batch_size=1, 
                          num_workers=4,
                         shuffle=False,
                         collate_fn=collate_fn)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


## Load the best models

In [ ]:
def inference(models, imgs, device):
    outs = None
    flip_outs = None
    flips = [[-1],[-2],[-2,-1]]
    for model in models:
        model.eval()
        if outs == None:
            outs = model(imgs.to(device).float()).detach()
        else:
            outs += model(imgs.to(device).float()).detach()
    
    # TTA
    for flip in flips: 
        flip_img = torch.flip(imgs, flip)
        tmp_outs = None
        for model in models:
            flip_out = model(flip_img.to(device).float()).detach()
            flip_out = torch.flip(flip_out, flip)
            if tmp_outs == None:
                tmp_outs = flip_out
            else:
                tmp_outs += flip_out  # soft-voting ensemble

        if flip_outs == None:
            flip_outs = tmp_outs
        else:
            flip_outs += tmp_outs
    
    outs += flip_outs
    
    return outs / ( len(models) * len(flips) + len(models) )
    

In [ ]:
def test(models, data_loader, device):
    size = 256
    transform = A.Compose([A.Resize(256, 256)])
    print('Start prediction.')
    file_name_list = []
    preds_array = np.empty((0, size*size), dtype=np.long)
    pbar = tqdm(enumerate(data_loader), total=len(data_loader), position=0, leave=True)
    with torch.no_grad():
        with zipfile.ZipFile(OUT_MASKS, 'w') as mask_out:
        for step, (imgs, image_infos) in pbar:
            imgs = torch.stack(imgs)
            outs = inference(models, imgs, device)
            oms = torch.argmax(outs, dim=1).detach().cpu().numpy()
                
                file_name = image_infos[0]['file_name'].split("/")
                file_name[0] += "_masks"
                file_name[1] = file_name[1][:-4]
                file_name = "/".join(file_name)

                m = cv2.imencode(".png", oms.squeeze())[1]
                mask_out.writestr(f"{file_name}.png", m)

            # resize (256 x 256)
            temp_mask = []
            for img, mask in zip(np.stack(imgs), oms):
                transformed = transform(image=img, mask=mask)
                mask = transformed['mask']
                temp_mask.append(mask)

            oms = np.array(temp_mask)
            oms = oms.reshape([oms.shape[0], size*size]).astype(int)

            preds_array = np.vstack((preds_array, oms))

            file_name_list.append([i['file_name'] for i in image_infos])
    print("End prediction.")
    file_names = [y for x in file_name_list for y in x]
    
    # return file_names, preds_array
    return preds_array

In [ ]:
MODEL_PATHS = ['/content/drive/MyDrive/Ensemble_0506/Nuree_pseudo_4/0_checkpoint.pt',
               '/content/drive/MyDrive/Ensemble_0506/Nuree_pseudo_4/1_checkpoint.pt',
               '/content/drive/MyDrive/Ensemble_0506/Nuree_pseudo_4/2_checkpoint.pt',
               '/content/drive/MyDrive/Ensemble_0506/Nuree_pseudo_4/3_checkpoint.pt',
               '/content/drive/MyDrive/Ensemble_0506/Nuree_pseudo_4/4_checkpoint.pt',
                ]


OUT_MASKS = f'/content/masks.zip'  # test 이미지에 대해 에측한 결과를 mask.zip으로 추출

In [ ]:
models=[]
for path in MODEL_PATHS:
    model = smp.DeepLabV3Plus('resnext50_32x4d', encoder_weights='swsl', classes=12)
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint)
    model.eval()
    model.to(device)
    models.append(model)
    
    del checkpoint

In [ ]:
preds = test(models, test_loader, device)  # 실행 끝나면 masks.zip 생성되어야 함.

Start prediction.



End prediction.


## submission.csv 생성

In [ ]:
# sample_submisson.csv 열기
submission = pd.read_csv('/content/drive/MyDrive/Trash_seg_0506/submission/sample_submission.csv', index_col=None)

# PredictionString 대입
for file_name, string in zip(file_names, preds):
    submission = submission.append({"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}, 
                                   ignore_index=True)

# submission.csv로 저장
submission.to_csv("./submission/pseudo.csv", index=False)